In [1]:
import pandas as pd
import os
from datetime import timedelta

1. Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [2]:
data = pd.read_csv(os.path.join(os.getcwd(), r'Python-6\keywords.csv'))
data.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [3]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}
def geo_classifier(row, geo_data):
    try:
        return [region for region, cities in geo_data.items() if set(row.keyword.split()).intersection(cities)][0]
    except IndexError:
        return 'undefined'

data['region'] = data.apply(geo_classifier, geo_data=geo_data, axis=1)
data[ data['region'] != 'undefined'].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


2. Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [6]:
data = pd.read_csv(os.path.join(os.getcwd(), r'Python-5\ml-latest-small\ratings.csv'))

def movie_rating_classifier(row):
    if row.rating <= 2:
        return 'низкий рейтинг'
    if row.rating <= 4:
        return 'средний рейтинг'
    if row.rating > 4.5:
        return 'высокий рейтинг'

data['class'] = data.apply(movie_rating_classifier, axis=1)
data.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг


3. Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [112]:
#оставим в df только пользователей, которые поставили 100 и более рейтингов
ratings_count = data.groupby('userId').agg({'rating':'count'}).reset_index()
users_over_100 = ratings_count[ ratings_count['rating'] >= 100 ]['userId'].tolist()
data_filtered_with_film_fans = data[ data['userId'].isin(users_over_100)]

#расчет lifetime отфильтованных пользователей в днях
data_filtered_with_film_fans.groupby('userId').agg({'timestamp':['min', 'max']}).apply(
    lambda x: timedelta(
        seconds=(int(x['timestamp']['max']) - int(x['timestamp']['min']))
    ).days, axis=1
).head()

userId
1       8
4     723
6       0
7     804
10      3
dtype: int64

4. Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

In [118]:
years = range(1950, 2011)

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

In [119]:
movies = pd.read_csv(os.path.join(os.getcwd(), r'Python-5\ml-latest-small\movies.csv'))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [129]:
def production_year(row):
    try:
        return [year for year in years if str(year) in row.title][0]
    except IndexError:
        return '1900'

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

In [130]:
movies['year'] = movies.apply(production_year, axis=1)

In [131]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [138]:
ratings = pd.read_csv(os.path.join(os.getcwd(), r'Python-5\ml-latest-small\ratings.csv'))
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()
joined.groupby('year').agg({'rating':'mean'}).reset_index().sort_values('rating', ascending=False).head(20)

,year,rating
7,1957,4.039535
4,1954,4.009191
12,1962,3.969466
2,1952,3.953125
22,1972,3.944293
14,1964,3.940160
24,1974,3.935622
17,1967,3.922572
25,1975,3.879121
8,1958,3.842424
